In [1]:
from subprocess import call
import re
import datetime as dt
from datetime import datetime
import time
import random
import json
import sys
import requests
import os
import wget
import pickle
import cv2


In [24]:
visitedList = "../Logs/facesVisited.data"
root ="../Data/"
faces = "faces"
face_cascade = cv2.CascadeClassifier('../haarcascades/haarcascade_frontalface_default.xml')
selected = "popularVines.pkl"
loopThreshold = 150000

In [27]:
def process_faces(frame):
        #print frame.shape
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        cv2.waitKey(20)
        face = []
        face = face_cascade.detectMultiScale(frame, 1.3, 5)
        if (len(face) > 0):
            cropped = []
            for i in range(len(face)):
                cropped.append(gray[face[i][1]:face[i][1]+face[i][3] , face[i][0]:face[i][0]+face[i][2]])
            return cropped
        else :
            return gray

def scaleSquare(image ,shape):
    resized = cv2.resize(image, shape, interpolation = cv2.INTER_AREA)
    return resized

def saveFaces(ImagePaths , faceFileName):
    faceFile = faceFileName
    faces =  []
    i = 0
    for path in ImagePaths:
        image = cv2.imread(path)
        cropped = process_faces(image)
        if len(cropped) > 0:
            for z in range(len(cropped)):
                if len(faces == 0):
                    faces = np.array(scaleSquare(cropped , (48 , 48)).flatten(), dtype=np.uint8)
                else:
                    np.append(faces , scaleSquare(cropped , (48 , 48)).flatten() , axis = 0)
    np.savetxt(faceFile, faces, delimiter=",")

def process_frame(frame, storeDir):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        cv2.waitKey(20)
        faces = []
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        if len(faces) > 0:
            print "Saving image at %s" % storeDir
            cv2.imwrite( storeDir , frame)
        return frame

def processVideo(videoPath , facesPath , postID):
    storeDir = facesPath + "/" + str(postID)
    if not os.path.exists(faceDir):
        os.makedirs(storeDir)
    cap = cv2.VideoCapture(videoPath)
    i = 0
    while True:
        ret, frame = cap.read()
        if ret:
            cv2.waitKey(20)
            if not os.path.exists(storeDir):
                os.makedirs(storeDir)
            imageName = storeDir + "/" + str(i) +".jpg"
            process_frame(frame, imageName)
            i += 1
        else:
            print "Done processing Video Number: %d "% postID
            break
    

def getVisited():
	visited = []
	try:
		f = open(visitedList, 'rb')
		visited = pickle.load(f)
	except IOError:
		f = open(visitedList,"a+")
		pickle.dump([],f)
	return visited


def updateVisited(visited):
	with open(visitedList, 'wb') as f:
		pickle.dump(visited, f)


def walkLevel1Dir(root):
	count = 0
	dirList = []
	filesList = []
	for path, dirs, files in os.walk(root):
		if count > 0:
			return dirList , fileList
		dirList = dirs
		fileList = files
		count = count + 1


def getPopularFile(rootDir):
		f = open(rootDir + '/popular.json' ,'r')
		data = json.load(f)
		return data

    
    
def getPopularPosts(popular , listFile):
    records = popular['data']['records']
    posts=[]
    for i in range (0 , len(records)):
        postID = records[i]['postId']
        loopCount = records[i]['loops']['count']
        if(loopCount > loopThreshold):
            posts.append(postID)
    
    with open(listFile, 'wb') as f:
        pickle.dump(posts, f)
    return posts



def getFaces(popular , faces):
        records = popular['data']['records']
        vidPaths=[]
        postIds=[]
        for i in range (0 , len(records)):
            postID = records[i]['postId']
            postURL = records[i]['videoDashUrl']
            if(postURL != None):
                vidURL = postURL.split('//')
                URLPaths = vidURL[1].split('?')
                vidPath = URLPaths[0]
                vidPaths.append(vidPath)
                postIds.append(postID)
                            
        return vidPaths, postIds

In [ ]:
if __name__ == '__main__':
    dirs,files = walkLevel1Dir(root)
    visited = getVisited()
    
    for d in dirs:
        if d not in visited:
            faceDir = root + d + "/" + faces
            selectedList = faceDir + "/" + selected

            if not os.path.exists(faceDir):
                os.makedirs(faceDir)
            
            dataRoot = root + d
            popular = getPopularFile(dataRoot)
            selectedPosts = getPopularPosts(popular , selectedList)

            paths, posts = getFaces(popular , faces)
            for i in range(len(posts)):
                if posts[i] in selectedPosts:
                    videoPath = root + d + "/videos/" + paths[i]
                    if os.path.exists(videoPath):
                        print "Processing Post ID %d with url %s"% (posts[i], paths[i])
                        processVideo(videoPath , faceDir , posts[i])

            visited.append(d)
            updateVisited(visited)
            break

Processing Post ID 1290475008904585216 with url v.cdn.vine.co/r/videos_dashhd/4D5FD5EE601290474988151123968_451eaff31cd.5.0.4132016358537318487.mp4
Saving image at ../Data/1450656001/faces/1290475008904585216/0.jpg
Saving image at ../Data/1450656001/faces/1290475008904585216/1.jpg
Saving image at ../Data/1450656001/faces/1290475008904585216/2.jpg
Saving image at ../Data/1450656001/faces/1290475008904585216/3.jpg
Saving image at ../Data/1450656001/faces/1290475008904585216/21.jpg
Saving image at ../Data/1450656001/faces/1290475008904585216/27.jpg
Saving image at ../Data/1450656001/faces/1290475008904585216/29.jpg
Saving image at ../Data/1450656001/faces/1290475008904585216/52.jpg
Saving image at ../Data/1450656001/faces/1290475008904585216/53.jpg
Saving image at ../Data/1450656001/faces/1290475008904585216/54.jpg
Saving image at ../Data/1450656001/faces/1290475008904585216/55.jpg
Saving image at ../Data/1450656001/faces/1290475008904585216/56.jpg
Saving image at ../Data/1450656001/faces